**1- Importation des librairies**

In [1]:
import pandas as pd
import glob
import numpy as np
import sys
import os
import re
from sklearn.ensemble import IsolationForest

**Monter le drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importer un fichier .Py contenant des fonctions utiles

In [3]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [4]:
from function_tools import *

In [5]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_brut= glob.glob('/content/drive/MyDrive/Data_M/bertsom25/*raw*')
#list_cluster= glob.glob('/content/drive/MyDrive/Data_M/bertsom25/*clean*')
list_array= glob.glob('/content/drive/MyDrive/Data_M/bertsom25/*encoded*')
file_info= '/content/drive/MyDrive/Data_M/bertsom25/bertsom25_clusters_info.txt'

In [6]:
# ordonner la liste des clusters
list_brut.sort(key=extr_num2)
list_array.sort(key=extr_num2)

**Vérifier l'extraction des fichiers**

In [ ]:
list_array

In [ ]:
list_brut

In [ ]:
file_info

'/content/drive/MyDrive/Data_M/bertsom25/bertsom25_clusters_info.txt'

**2- Lire le contenu des clusters**

In [8]:
brut_cluster= read_cluster(list_brut)

**3- Transformer les listes en dataframe**

In [9]:
def init_df(brut_cluster):
  liste_df=[]
  for i in range(len(brut_cluster)):
    liste_df.append(pd.DataFrame(list(brut_cluster[i]), columns = ['tweet']))
  return liste_df


In [10]:
liste_df=init_df(brut_cluster)

In [11]:
liste_df[24].head()

,tweet
0,gaston lagaffe au bureau : dsl pour la ptite g...
1,Un gars de type méditerranéen (oui maghrébin) ...
2,L'humour noir dont il parle c'est du viol sur ...
3,Pour vous sale negro on doit subir si non on a...
4,ça doit être le genre de mec à jouir en entend...


**4- Mettre les vecteurs dans une liste**

In [12]:
liste_vec=[]
for i in range(len(list_array)):
  liste_vec.append(np.load(list_array[i]))

In [13]:
liste_vec[14].shape

(97, 768)

**5- Entrainement avec Isolation Forest**

**A- Variante contamination fixe**

In [14]:
def train_if (dataset, cont):
  models=[]
  for i in range(len(dataset)):
    if_model=IsolationForest(n_jobs=-1, random_state=42, contamination=cont)
    models.append(if_model.fit(dataset[i]))
  return models

In [15]:
# Entrainer le modèle
%%time
models = train_if(liste_vec, 0.15)

CPU times: user 9.24 s, sys: 533 ms, total: 9.77 s
Wall time: 9.48 s


In [58]:
# mettre les prédictions dans une liste
def pred(dataset,model):
  pred=[]
  for i in range(len(dataset)):
    pred.append(model[i].predict(dataset[i]))
  return pred

In [17]:
pred= pred(liste_vec, models)

In [63]:
# ajouter une colonne pred à chaque dataframe de cluster
def pred_to_df(liste, pred):
  for i in range(len(liste)):
    liste[i]['pred']= pred[i]

In [19]:
pred_to_df(liste_df)

In [20]:
liste_df[14].head(10)

,tweet,pred
0,Sale noir vas\n,1
1,Ya quoi sale noir\n,1
2,Jsui mort\n,1
3,Bah le babouin\n,1
4,je préfère banania\n,1
5,Negre de maison\n,1
6,c toi sale noir\n,1
7,Nègre brésilien triangulaire\n,1
8,J’ai menti ?\n,1
9,oh ptn sale noir\n,1


**Extraction des clusters pertinent**

In [21]:
def extract_pourcentage(filename):
  pourcentage_haineux = []
  with open(filename, "r") as f:
    lines = f.read().split('\n\n')
  for line in lines[:-1]: 
    hper = re.findall('(\d*.\d*)% de', line)
    hper = float(hper[0]) if len(hper) > 0 else 0
    pourcentage_haineux.append(hper)
  return pourcentage_haineux

In [22]:
pourcentage_haineux= extract_pourcentage(file_info)

In [23]:
def filtre_cluster_pertinent(cont):
  pertinent_cluster=[]
  for i in range(len(cont)):
    if cont[i]!= 0 and cont[i] < 20 or cont[i] > 80 :
      pertinent_cluster.append(i)
  return pertinent_cluster

In [24]:
pertinent_cluster=filtre_cluster_pertinent(pourcentage_haineux)

In [25]:
pertinent_cluster

[0, 12, 13, 14, 17, 19]

In [26]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def separate_outliers(liste_df):
  outliers=[]
  clusters_clean=[]
  for i,n in enumerate(pertinent_cluster):
    outliers.append(liste_df[n]['tweet'].loc[liste_df[n]['pred']==-1])
    clusters_clean.append(liste_df[n]['tweet'].loc[liste_df[n]['pred']==1])
    #outlier_index=list(outliers[i].index)
    print("cluster"+str(n)+":"+"\n")
    print(liste_df[n]['pred'].value_counts())
    print("\n")
  return outliers, clusters_clean

In [ ]:
outliers_fixe, clusters_clean_fixe= separate_outliers(liste_df)

In [28]:
def save_cluster(liste, file, repo):
  for i,n in enumerate(pertinent_cluster):
    with open(f"drive/MyDrive/"+repo+file+f"_{n}", 'w') as f:
      for line in liste[i]:
        f.write(line)

In [29]:
save_cluster(outliers_fixe, "fixe_outliers", "outliers_bert_som25/")

In [30]:
save_cluster(clusters_clean_fixe, "fixe_clean","clean_bert_som25/")

**B- Variante contamination variable**

In [31]:
liste_df_var=init_df(brut_cluster)

In [ ]:
pourcentage_haineux

In [33]:
pertinent_cluster

[0, 12, 13, 14, 17, 19]

In [34]:
def train_if_var (dataset, cont):
  models=[]
  for i in range(len(dataset)):
    if cont[i] < 20 and cont[i]!= 0:
      if_model=IsolationForest(n_jobs=-1, random_state=42, contamination=cont[i]/100)
      models.append(if_model.fit(dataset[i]))
    elif cont[i]> 80:
      if_model=IsolationForest(n_jobs=-1, random_state=42, contamination=1-(cont[i]/100))
      models.append(if_model.fit(dataset[i]))
  return models

In [35]:
models_var= train_if_var(liste_vec, pourcentage_haineux)

In [36]:
len(models_var)

6

In [37]:
def pred_var(dataset,model):
  pred=[]
  for i,n  in enumerate(pertinent_cluster):
    pred.append(model[i].predict(dataset[n]))
  return pred

In [38]:
prediction = pred_var(liste_vec,models_var)

In [ ]:
len(prediction)

6

In [39]:
# ajouter une colonne pred à chaque dataframe de cluster
for i, n in enumerate(pertinent_cluster):
  liste_df_var[n]['pred']= prediction[i]

In [ ]:
liste_df_var[0]['pred'].value_counts()

 1    182
-1     43
Name: pred, dtype: int64

In [ ]:
outliers_var, clusters_clean_var= separate_outliers(liste_df_var)

In [41]:
save_cluster(outliers_var, "var_outliers", "outliers_bert_som25/")

In [42]:
save_cluster(clusters_clean_var, "var_clean","clean_bert_som25/" )

**C- Variante contamination automatique**

In [59]:
def train_if_auto (dataset):
  models=[]
  for i in range(len(dataset)):
    if_model=IsolationForest(n_jobs=-1, random_state=42)
    models.append(if_model.fit(dataset[i]))
  return models

In [60]:
models_auto = train_if_auto(liste_vec)

In [61]:
liste_df_auto=init_df(brut_cluster)

In [62]:
pred_auto = pred(liste_vec, models_auto)

In [64]:
pred_to_df(liste_df_auto, pred_auto)

In [ ]:
liste_df_auto

In [66]:
outliers_auto, clusters_clean_auto= separate_outliers(liste_df_auto)

cluster0:

 1    224
-1      1
Name: pred, dtype: int64


cluster12:

 1    138
-1      4
Name: pred, dtype: int64


cluster13:

 1    129
-1      4
Name: pred, dtype: int64


cluster14:

 1    95
-1     2
Name: pred, dtype: int64


cluster17:

 1    68
-1     5
Name: pred, dtype: int64


cluster19:

 1    350
-1      4
Name: pred, dtype: int64




In [67]:
save_cluster(outliers_auto, "auto_outliers", "outliers_bert_som25/")

In [68]:
save_cluster(clusters_clean_auto, "auto_clean", "clean_bert_som25/")